In [1]:
!pip install kmodes

In [4]:
import numpy as np
import pandas as pd
from scipy.special import logsumexp
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score
from kmodes.kmodes import KModes


In [5]:
class BernoulliMixtureSEM:
    def __init__(self, n_components, max_iter, tol=1e-3, n_samples_per_component=1):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.n_samples_per_component = n_samples_per_component

    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                break
            # Check for convergence
            if np.abs(self.logL - self.old_logL) < self.tol:
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()

    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1 / self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()

    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k, :] /= sum_over_features[k]

    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(self.n_samples, self.n_components))
        Z = logsumexp(np.log(self.pi[None, :]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:, k] - Z)
        # Sample latent variables
        for i in range(self.n_samples):
            gamma[i, :] = np.random.multinomial(1, gamma[i, :])
        return gamma

    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1 - x, 1 - self.mu)
        return log_bernoullis

    def get_save_single(self, x, mu):
        mu_place = np.where(mu <= 1e-15, 1e-15, mu)
        return np.tensordot(x, np.log(mu_place), (1, 1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)

    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:, None]

    def get_pi(self):
        self.pi = self.Neff / self.n_samples

    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)

    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None, :]) + log_bernoullis, axis=1)

    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))

    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)

    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [9]:
# Load the Zoo dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data"
columns = ["animal_name", "hair", "feathers", "eggs", "milk", "airborne", "aquatic",
           "predator", "toothed", "backbone", "breathes", "venomous", "fins", "legs",
           "tail", "domestic", "catsize", "type"]

zoo = pd.read_csv(url, names=columns)
X = zoo.drop(["animal_name", "type"], axis=1)
y = zoo["type"]

In [10]:
# Encode categorical features as integers
label_encoders = {col: LabelEncoder().fit(X[col]) for col in X.columns}
for col, le in label_encoders.items():
    X[col] = le.transform(X[col])

# Ensure the dataset is binary for the Bernoulli Mixture Model
X_binary = (X > 0).astype(int)

In [11]:
# Initialize and fit the Bernoulli Mixture model with SEM
sem_model = BernoulliMixtureSEM(n_components=7, max_iter=100)
sem_model.fit(X_binary.values)
sem_predictions = sem_model.predict(X_binary.values)

# Initialize and fit the K-Modes model
km = KModes(n_clusters=7, init='Huang', n_init=5, verbose=1)
km.fit(X)
km_predictions = km.predict(X)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 12, cost: 150.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 22, cost: 161.0
Run 2, iteration: 2/100, moves: 0, cost: 161.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 9, cost: 152.0
Run 3, iteration: 2/100, moves: 0, cost: 152.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 7, cost: 156.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 20, cost: 150.0
Run 5, iteration: 2/100, moves: 5, cost: 150.0
Best run was number 1


In [12]:
# Evaluate clustering performance
fmi_sem = fowlkes_mallows_score(y, sem_predictions)
ari_sem = adjusted_rand_score(y, sem_predictions)
nmi_sem = normalized_mutual_info_score(y, sem_predictions)

fmi_km = fowlkes_mallows_score(y, km_predictions)
ari_km = adjusted_rand_score(y, km_predictions)
nmi_km = normalized_mutual_info_score(y, km_predictions)

In [13]:
# Print the results
print("Bernoulli Mixture Model with SEM:")
print(f"FMI: {fmi_sem}, ARI: {ari_sem}, NMI: {nmi_sem}")

print("\nK-Modes:")
print(f"FMI: {fmi_km}, ARI: {ari_km}, NMI: {nmi_km}")

Bernoulli Mixture Model with SEM:
FMI: 0.6655506092840433, ARI: 0.5734476270123353, NMI: 0.6896782659344508

K-Modes:
FMI: 0.6298860033747976, ARI: 0.529475551586335, NMI: 0.7272630630385678
